In [6]:
import os
import subprocess

def reproject_shapefile(input_path, output_path, target_srs="EPSG:4326"):
    """
    Reprojects a shapefile to a specified spatial reference system (e.g., EPSG:4326).
    
    Parameters:
        input_path (str): Path to the input shapefile (.shp).
        output_path (str): Path to save the reprojected shapefile.
        target_srs (str): Target spatial reference system (e.g., "EPSG:4326").
    """
    try:
        subprocess.run([
            "ogr2ogr",
            "-t_srs", target_srs,
            "-f", "ESRI Shapefile",
            output_path,
            input_path
        ], check=True)
        print(f"Reprojected shapefile saved at {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error reprojecting shapefile {input_path}: {e}")

def convert_shapefile_to_pmtiles(shapefile_path, pmtiles_path, min_zoom=10, max_zoom=15):
    """
    Converts a shapefile to PMTiles format using GDAL's ogr2ogr.
    
    Parameters:
        shapefile_path (str): Path to the shapefile (.shp).
        pmtiles_path (str): Path to save the PMTiles output.
        min_zoom (int): Minimum zoom level for PMTiles.
        max_zoom (int): Maximum zoom level for PMTiles.
    """
    try:
        subprocess.run([
            "ogr2ogr",
            "-dsco", f"MINZOOM={min_zoom}",
            "-dsco", f"MAXZOOM={max_zoom}",
            "-f", "PMTiles",
            pmtiles_path,
            shapefile_path
        ], check=True)
        print(f"PMTiles created at {pmtiles_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {shapefile_path}: {e}")

def create_pmtiles_for_directory(shapefile_dir, pmtiles_dir, min_zoom=5, max_zoom=12):
    """
    Scans a directory of shapefiles and generates PMTiles for each.
    
    Parameters:
        shapefile_dir (str): Directory containing shapefiles (including subdirectories).
        pmtiles_dir (str): Directory to save the generated PMTiles files.
        min_zoom (int): Minimum zoom level for PMTiles.
        max_zoom (int): Maximum zoom level for PMTiles.
    """
    # Ensure the PMTiles output directory exists
    os.makedirs(pmtiles_dir, exist_ok=True)
    
    # Gather all shapefiles recursively
    shapefiles = []
    for root, dirs, files in os.walk(shapefile_dir):
        for file in files:
            if file.endswith(".shp"):
                shapefiles.append(os.path.join(root, file))
    
    print(f"Found {len(shapefiles)} shapefiles to process.")

    # Process each shapefile
    for shapefile_path in shapefiles:
        # Define the paths for the intermediate reprojected shapefile and the final PMTiles
        reprojected_shapefile_path = os.path.join(pmtiles_dir, f"reprojected_{os.path.basename(shapefile_path)}")
        pmtiles_name = f"{os.path.splitext(os.path.basename(shapefile_path))[0]}.pmtiles"
        pmtiles_path = os.path.join(pmtiles_dir, pmtiles_name)
        
        # Step 1: Reproject the shapefile to EPSG:4326
        reproject_shapefile(shapefile_path, reprojected_shapefile_path)

        # Step 2: Convert the reprojected shapefile to PMTiles
        convert_shapefile_to_pmtiles(reprojected_shapefile_path, pmtiles_path, min_zoom, max_zoom)

        # Optional: Remove the reprojected shapefile to save space
        os.remove(reprojected_shapefile_path)
        print(f"Deleted temporary file {reprojected_shapefile_path}")

# Usage
shapefile_directory = "HPMS_NJ_Roads shp"
pmtiles_directory = "pmtiles"

create_pmtiles_for_directory(shapefile_directory, pmtiles_directory)


Found 1 shapefiles to process.


Warning 1: Value 50.1599999999999966 of field MP_END of feature 6 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 50.0900000000000034 of field MEASURED_L of feature 6 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 50.0900000000000034 of field PARENT_MP_ of feature 6 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 77.9680000000000035 of field PARENT_MP_ of feature 7 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 73.7879999999999967 of field PARENT_MP1 of feature 7 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 73.2780000000000058 of field PARENT_MP_ of feature 8 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 71.6280000000000001 of field PARENT_MP1

Reprojected shapefile saved at pmtiles/reprojected_HPMS_NJ_Roads_shp.shp


Warning 1: The output driver does not natively support Date type for field YEAR_ACTIV. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not natively support Date type for field YEAR_RETIR. Misconversion can happen. -mapFieldType can be used to control field type conversion.


PMTiles created at pmtiles/HPMS_NJ_Roads_shp.pmtiles
Deleted temporary file pmtiles/reprojected_HPMS_NJ_Roads_shp.shp
